# Set Membership Coursework

In [32]:
import random
import string
import timeit
from bitarray import bitarray

## 1. Implementeation

### Sequential Search 

In [33]:
class SequentialSearchSet():
    
    def __init__(self):
        self.items = []           
        
    def insertElement(self, element):
        self.items.append(element)
        return True   

    def searchElement(self, element):
        found = False     
        for item1 in self.items:
            if item1 == element:
                found = True
            else:
                found = False
        return found   

Time Complexity: 
- Insert an element is O(n^2)
- Search an element is O(n)

Space Complexity: 
- O(n)

### Binary Search Tree

In [34]:
class Node:
    
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

    def get(self, element):
        if self.value == element:
            return True
        elif element < self.value and self.left:
            return self.left.get(element)
        elif element > self.value and self.right:
            return self.right.get(element)
        return False

    def put(self, element):
        if self.value == None:
            self.value = element
            return True
        elif element == self.value:
            self.value = element
            return True
        elif element < self.value:
            if self.left is None:
                self.left = Node(element)
                return True
            else: 
                self.left.put(element)
        elif element > self.value:
            if self.right is None:
                self.right = Node(element)
                return True
            else: 
                self.right.put(element)
        return False

class BinarySearchTreeSet:
    
    def __init__(self):
        self.bst = Node(None)

    def insertElement(self, element):
        inserted = False
        inserted = self.bst.put(element) 
        return inserted

    def searchElement(self, element):     
        found = False 
        found = self.bst.get(element)
        return found

### Balanced Search Tree

In [35]:
# Red-Black Search Tree

class BalancedSearchTreeSet:
    R = True
    B = False
    # R refers to Red; B refers to Black
    
    class Nodes:
        def __init__(self, val, colour):
            self.val = val
            self.colour = colour
            self.left = None
            self.right = None
            
    def __init__(self):
        self.root = None
        
    def searchElement(self, element):
        return self.search(self.root, element)
    
    def search(self, node, val):
        if node is None:
            return False
        else:
            if node.val == val:
                return True
            elif node.val > val:
                return self.search(node.left, val)
            else:
                return self.search(node.right, val)
    
    def insertElement(self, element):
        self.root = self.insert(self.root, element)
        self.root.colour = self.B


    def insert(self, node, val):
        if node is None:
            return self.Nodes(val, self.R)

        if val < node.val:
            node.left = self.insert(node.left, val)
        elif val > node.val:
            node.right = self.insert(node.right, val)
        else:
            # value already exists, do nothing
            return node

        # perform rotations and colour flips
        if self.isR(node.right) and not self.isR(node.left):
            node = self.leftRotate(node)
        if self.isR(node.left) and self.isR(node.left.left):
            node = self.rightRotate(node)
        if self.isR(node.left) and self.isR(node.right):
            self.flipColour(node)

        return node

    def isR(self, node):
        if node is None:
            return False
        return node.colour == self.R
    
    def leftRotate(self, node):
        x = node.right
        node.right = x.left
        x.left = node
        x.colour = node.colour
        node.colour = self.R
        return x
    
    def rightRotate(self, node):
        x = node.left
        node.left = x.right
        x.right = node
        x.colour = node.colour
        node.colour = self.R
        return x
    
    def flipColour(self, node):
        node.colour = not node.colour
        node.left.colour = self.B
        node.right.colour = self.B

Time Complexity: 

- Insert an element is O(log n)
- Search an element is O(log n)

Space Complexity: 

- O(n)

### Bloom Filter

In [36]:
class BloomFilterSet:
    
    def __init__(self):
        self.HashFunctions = None
        self.hashNum = 0
        self.size = 0
        self.bitArray = None
    
    def initVariables(self, ItemNum, FPProb, HashFunctions):
        # ItemNum : int, Number of items expected to be stored in bloom filter
        # FPProb : float, False Positive probability in decimal
        # HashFunctions: list, number of hash functions to use
        self.HashFunctions = HashFunctions
        self.hashNum = len(HashFunctions)
        # Total size of bit array
        self.size = self.getSize(ItemNum, FPProb)
        # Bit array of given size, and initialize all bits as 0 (False)
        self.bitArray = bitarray(self.size)
        self.bitArray.setall(0)

    def insertElement(self, element):
        inserted = False
        tempIndex = []
        for function in self.HashFunctions:
            index = function(element) % self.size
            self.bitArray[index] = True
            tempIndex.append(self.bitArray[index])
        if False not in tempIndex:
            inserted = True
        return inserted

    def searchElement(self, element):
        found = False
        count = 0
        for function in self.HashFunctions:
            index = function(element) % self.size
            if not self.bitArray[index] == False:
                count += 1
        if count == len(self.HashFunctions):
            found = True
        return found

    def getSize(self, n, p):
        # m = -(n * lg(p)) / (lg(2)^2)
        # n : number of items expected to be stored in filter
        # p : False Positive probability in decimal
        m = -(n * self.myLog(p)) / (self.myLog(2) ** 2)
        return int(m)
    
    def myLog(self, n):
        if n <= 0:
            raise ValueError("logarithm is undefined for non-positive values")
        if n == 1:
            return 0
        if n < 1:
            # log(xn) = -log(1/n) for n < 1
            return -self.myLog(1 / n)
        # Use the taylor series approximation for log(n) where |n-1| < 1
        taylor = (n - 1) / float(n + 1)
        term = taylor
        total = taylor
        num = 1
        while abs(term) > 1e-15:
            num += 2
            term *= taylor * taylor / float(num * (num - 1))
            total += term
        return 2 * total

In [37]:
def builtinHash(string):
    return hash(string)

def bernsteinHash(string):
    hashValue = 5381
    for char in string:
        hashValue = ((hashValue << 5) + hashValue) + ord(char)
    return hashValue

def fnvHash(string):
    hashValue = 2166136261
    for char in string:
        hashValue = (hashValue * 16777619) ^ ord(char)
    return hashValue

def jenkinsHash(string):
    hashValue = 0
    for char in string:
        hashValue += ord(char)
        hashValue += (hashValue << 10)
        hashValue ^= (hashValue >> 6)
    hashValue += (hashValue << 3)
    hashValue ^= (hashValue >> 11)
    hashValue += (hashValue << 15)
    return hashValue

def djb2Hash(string):
    hashValue = 5381
    for char in string:
        hashValue = ((hashValue << 5) + hashValue) + ord(char)
    return hashValue

def sdbmHash(string):
    hashValue = 0
    for char in string:
        hashValue = ord(char) + (hashValue << 6) + (hashValue << 16) - hashValue
    return hashValue

def fnv1aHash(string):
    hashValue = 2166136261
    for char in string:
        hashValue ^= ord(char)
        hashValue *= 16777619
    return hashValue

In [38]:
Alphabets = list(string.ascii_letters)
randomword = []
for a in range(10000):
    temp = ''.join(random.choice(Alphabets) for t in range(random.randint(100, 200)))
    randomword.append(temp)

functions = [builtinHash, bernsteinHash, fnvHash, jenkinsHash, 
             djb2Hash, sdbmHash, fnv1aHash]
bf = BloomFilterSet()
bf.initVariables(10000, 0.01, functions)
testnum = 0
for ws in randomword:
    if bf.searchElement(ws) == True:
        testnum += 1

print("The False-Positive-Rate: ", testnum / len(randomword))

The False-Positive-Rate:  0.0


Time Complexity: 
- Creating the Bloom filter is O(kn)
- Insert and Search an element: O(k)
- k is the number of hash functions and n is the number of elements

Space Complexity: 
- O(kb), where k is the number of hash functions and b is the bits number

## 2. Experimentally Evaluation

### Using Real Data

In [39]:
def ReadFile(filepath):
    with open(filepath) as f:
        for line in f:
            words = line.strip().split()
    return words

In [40]:
words1 = ReadFile("./data/testfiles/test1-mobydick.txt")
words2 = ReadFile("./data/testfiles/test2-warpeace.txt")
words3 = ReadFile("./data/testfiles/test3-dickens.txt")

files = {"'test1-mobydick.txt'": words1, "'test2-warpeace.txt'": words2,
         "'test3-dickens.txt'": words3}
print(len(words1), len(words2), len(words3))

209329 564236 5149661


In [41]:
wordsforsearch = []
with open("./data/testfiles/test-search.txt") as fsearch:
    for line in fsearch:
        wordsforsearch.append(line.strip())
        
len(wordsforsearch)

544

In [42]:
hashFunctions = [builtinHash, bernsteinHash, fnvHash, jenkinsHash, 
                 djb2Hash, sdbmHash, fnv1aHash]

#### The total amount of time of all insert operations

In [43]:
def InsertTime(classname, datadict):
    classlist = []
    for key in datadict.keys():
        algorithm = classname()
        InsertStart = timeit.default_timer()
        for word in datadict[key]:
            algorithm.insertElement(word)
        InsertEnd = timeit.default_timer()
        InsertTime = InsertEnd - InsertStart
        print("Insert time taken for", key, "is :", InsertTime)
        classlist.append(algorithm)
    return classlist

In [44]:
# Sequential Search
SEQs = InsertTime(SequentialSearchSet, files)

Insert time taken for 'test1-mobydick.txt' is : 0.16921989999991638
Insert time taken for 'test2-warpeace.txt' is : 0.422850599999947
Insert time taken for 'test3-dickens.txt' is : 3.8224537000000964


In [45]:
# Binary Search Tree
BSTs = InsertTime(BinarySearchTreeSet, files)

Insert time taken for 'test1-mobydick.txt' is : 2.865964900000108
Insert time taken for 'test2-warpeace.txt' is : 6.461879699999827
Insert time taken for 'test3-dickens.txt' is : 55.01033860000007


In [46]:
# Balanced Search Tree
RBTs = InsertTime(BalancedSearchTreeSet, files)

Insert time taken for 'test1-mobydick.txt' is : 6.675154699999894
Insert time taken for 'test2-warpeace.txt' is : 16.899827799999912
Insert time taken for 'test3-dickens.txt' is : 698.9597761


In [47]:
# Bloom Filter
BFs = []
for key in files.keys():
    algorithm = BloomFilterSet()
    algorithm.initVariables(len(files[key]), 0.01, hashFunctions)
    InsertStart = timeit.default_timer()
    for word in files[key]:
        algorithm.insertElement(word)
    InsertEnd = timeit.default_timer()
    InsertTime = InsertEnd - InsertStart
    print("Insert time taken for", key, "is :", InsertTime)
    BFs.append(algorithm)

Insert time taken for 'test1-mobydick.txt' is : 3.1934015000001637
Insert time taken for 'test2-warpeace.txt' is : 8.451915900000131
Insert time taken for 'test3-dickens.txt' is : 78.55494840000028


#### The total amount of time of search operations

In [48]:
def SearchTime(objects, searchdata):
    for obj in objects:
        SearchStart = timeit.default_timer()
        for ws in searchdata:
            obj.searchElement(ws)
        SearchEnd = timeit.default_timer()
        SearchTime = SearchEnd - SearchStart
        print("Search time taken for 'test-search.txt' in file", 
              objects.index(obj)+1, "is :", SearchTime)

In [49]:
# Sequential Search Tree
SearchTime(SEQs, wordsforsearch)

Search time taken for 'test-search.txt' in file 1 is : 9.931449599999723
Search time taken for 'test-search.txt' in file 2 is : 27.314137600000322
Search time taken for 'test-search.txt' in file 3 is : 275.07406850000007


In [50]:
# Binary Search Tree
SearchTime(BSTs, wordsforsearch)

Search time taken for 'test-search.txt' in file 1 is : 0.007458699999915552
Search time taken for 'test-search.txt' in file 2 is : 0.007272499999999127
Search time taken for 'test-search.txt' in file 3 is : 0.006699300000036601


In [51]:
# Balanced Search Tree
SearchTime(RBTs, wordsforsearch)

Search time taken for 'test-search.txt' in file 1 is : 0.006438599999910366
Search time taken for 'test-search.txt' in file 2 is : 0.006815900000219699
Search time taken for 'test-search.txt' in file 3 is : 0.0067853999998988


In [52]:
# Bloom Filter
SearchTime(BFs, wordsforsearch)

Search time taken for 'test-search.txt' in file 1 is : 0.009318900000380381
Search time taken for 'test-search.txt' in file 2 is : 0.009477800000240677
Search time taken for 'test-search.txt' in file 3 is : 0.009106699999847478


### Using Synthetic Data

### different conditions:
- Duplicate cases
- Longer String length
- Adding numbers and symbols in each string
- String list in ascending/descending order
- Missing values
- ......

In [53]:
class TestDataGenerator():

    def __init__(self):
        self.syntheticData = []
        self.alphabets = list(string.ascii_letters)
        self.size = 0

    def setSize(self, size):
        self.size = size
    
    def generateData(self, strLen, percentage):
#         data, data1, data2 = [""] * self.size, [""] * self.size, [""] * self.size
        data, data1, data2 = [], [], []
        charlist = string.printable
#         i, j, k = 0, 0, 0
        while len(data) < self.size and len(data1) < self.size and len(data2) < self.size:
            # Normal Case
            temp = ''.join(random.choice(self.alphabets) for _ in range(random.randint(5, 10)))
            if temp not in data:
                data.append(temp)
            # Long String   
            temp1 = ''.join(random.choice(self.alphabets) for _ in range(strLen))
            if temp1 not in data1:
                data1.append(temp1)
            # Special String
            temp2 = ''.join(random.choice(charlist) for _ in range(random.randint(5, 10)))
            if temp2 not in data2:
                data2.append(temp2)
        print(0,1,2)
#         for i in range(self.size):
#             # Normal Case
#             data[i] = ''.join(random.choice(self.alphabets) for _ in range(random.randint(2, 15)))
#             # Long String   
#             data1[i] = ''.join(random.choice(self.alphabets) for _ in range(strLen))
#             # Special String
#             data2[i] = ''.join(random.choice(charlist) for _ in range(random.randint(2, 15)))
            
        # Ascending Order
        data3 = self.mergeSort(data.copy())
        # Descending Order
        data4 = data3[::-1]
        print(3,4)
        data5, data6 = data.copy(), data.copy()
        # String with missing values
        num6 = round(self.size * 0.1)
        temp6 = data6[ : num6]
        for _ in range(round(percentage * self.size)):
            index = random.choice(range(num6, self.size))
            # String with missing values
            data5[index] = None
            # String with duplicate values
            data6[index] = random.choice(temp6)
        
        self.syntheticData.append(data)
        self.syntheticData.append(data1)
        self.syntheticData.append(data2)
        self.syntheticData.append(data3)
        self.syntheticData.append(data4)
        self.syntheticData.append(data5)
        self.syntheticData.append(data6)
        return self.syntheticData
    
    def mergeSort(self, arr):
        n = len(arr)
        size = 1
        while size < n:
            for i in range(0, n, 2*size):
                left = arr[i : i+size]
                right = arr[i+size : i+2*size]
                merged = []
                l = r = 0
                while l < len(left) and r < len(right):
                    if left[l] < right[r]:
                        merged.append(left[l])
                        l += 1
                    else:
                        merged.append(right[r])
                        r += 1
                merged += left[l:]
                merged += right[r:]
                arr[i : i+len(merged)] = merged
            size *= 2
        return arr

In [60]:
# missing value checker

def isNone(list):
    i = 0
    lst_iter = iter(list)
    while True:
        element = next(lst_iter,"end")
        if element == "end":
            break
        if element is None:
            i += 1
    print("The file contains %d None values" % i)

testList = [1,2,3,None,4,5,6,7,8,9,None]

isNone(words1)
isNone(words2)
isNone(words3)
isNone(testList)

The file contains 0 None values
The file contains 0 None values
The file contains 0 None values
The file contains 2 None values


In [54]:
toGenerate = TestDataGenerator()
toGenerate.setSize(1000)
SyntheticDataList = toGenerate.generateData(50, 0.6)
SyntheticDataList

0 1 2
3 4


[['UWezePo',
  'FGHdlaMEmR',
  'saPrJeOq',
  'dfJqCmKnI',
  'xGDhhJtlzx',
  'VZrmixV',
  'ThFLeE',
  'dYnQV',
  'pTIhwtirA',
  'aKwkbyiG',
  'ceNhxWUw',
  'pfPDbw',
  'pBaaurG',
  'qDaUNwsmls',
  'iwTwwC',
  'RSegNQtS',
  'JTsdLeciSX',
  'tjOdcin',
  'dEFqi',
  'mEiPfOm',
  'wurZdUrOEn',
  'nNYsd',
  'qMLTpgv',
  'XBbfrAYi',
  'yVHRhLaXnZ',
  'wqDVDnz',
  'mpzxgraSX',
  'jolzne',
  'wBXDAkrakq',
  'TqggEIVOLv',
  'xOaWJEXY',
  'XVNXvPJjBN',
  'qUQhq',
  'TUekMPndM',
  'ohAMFyk',
  'UcUZSAhS',
  'oCApQ',
  'aibbTtCVW',
  'UZQGMwU',
  'JotiXuBc',
  'ZGMLSByRJ',
  'FFONFXC',
  'ForHigCrtw',
  'QEkwxLBY',
  'aOIosBFe',
  'TwtMia',
  'xYBLZl',
  'VJLnaRt',
  'TLTGaZPbzx',
  'EkTRnF',
  'lyBEC',
  'rpQSn',
  'vqZwHHs',
  'SEgnfmouN',
  'IDAFNLqHq',
  'PUzsxp',
  'yPitBZSAqU',
  'uVbYRj',
  'MDsSj',
  'JTwfg',
  'dZGHpx',
  'KYQaENaH',
  'LGflkmOI',
  'zFluN',
  'qfGPFwHci',
  'vzWWEkLQy',
  'GqOwPPy',
  'ehzmRHFJoT',
  'VtmiZiDyHm',
  'kMZKI',
  'BGFbBaLzZS',
  'jjmfNNFP',
  'iysErViOE',
  '

In [55]:
SyntheticDataNames = ["Normal", "Long String", "Special String", "Ascending Order", "Descending Order", 
                      "String with missing values", "String with duplicate values"]

In [56]:
synSearchList = []
for datalist in SyntheticDataList[0 : 3]:
    num = round(len(datalist) * 0.3)
    templist = []
    while len(templist) < num:
        tempdata = random.choice(datalist)
        if tempdata not in templist:
            templist.append(tempdata)
    synSearchList.append(templist)

for _ in range(4):
    synSearchList.append(synSearchList[0].copy())
synSearchList

[['pwmkyAklSi',
  'iSUAEXTe',
  'KEDKKPjLzQ',
  'ugFQzYeqIl',
  'UFDtFe',
  'egKvqhf',
  'wZDlNa',
  'WSyYVuSv',
  'OfrdkZrY',
  'dvkoqSt',
  'BPzHlnNZD',
  'bzbyS',
  'XtHSBty',
  'AHrKIcDAHj',
  'WlMPQ',
  'QZDRt',
  'HvBGsYxsb',
  'PZotc',
  'yPitBZSAqU',
  'qXZdH',
  'xgvErKNQpX',
  'bjYLDsGdaq',
  'tvUdaSi',
  'rzRnCKd',
  'apdWjW',
  'eqxKMBhER',
  'JwYws',
  'kGvyndn',
  'XGUqHp',
  'ERbgT',
  'VwqFFkc',
  'onRVzugydh',
  'CnfxU',
  'obuOdh',
  'VrGwpTRAXm',
  'AyQAhGvFIh',
  'KYlWFQMLmB',
  'mMWLJTQ',
  'VIJRxCZsl',
  'LvWGjATNiL',
  'wRlWNNaxA',
  'ZuHGLGipqh',
  'FsHjYN',
  'XEknB',
  'VJLnaRt',
  'jJJDbODyvp',
  'PVthc',
  'fCuFDZC',
  'ruJIbWB',
  'pImhIuzHQf',
  'Figpikq',
  'QibpRzAY',
  'iTbEU',
  'eYdin',
  'dfJqCmKnI',
  'kPbvPT',
  'jWbIsP',
  'SGEco',
  'YrxeNRzo',
  'BrOQU',
  'dHoll',
  'jjmfNNFP',
  'vmVFCZp',
  'pUdJzl',
  'ACalf',
  'MCkTjN',
  'UjwcJA',
  'ehzmRHFJoT',
  'RsxmexLtM',
  'TUekMPndM',
  'ndCFM',
  'wBiHn',
  'qFLxqiTqAT',
  'iLosFFEHft',
  'bedJka

#### Insert Time Taken

In [57]:
def InsertTime2(classname, datalist):
    algorithm = classname()
    InsertStart = timeit.default_timer()
    for word in datalist:
        algorithm.insertElement(word)
    InsertEnd = timeit.default_timer()
    InsertTime = InsertEnd - InsertStart
    print("Insert time taken is :", InsertTime)
    return algorithm

In [58]:
SEQSyn, BSTSyn, RBTSyn, BTSyn = [], [], [], []
for i in range(len(SyntheticDataList)):
    print("For", SyntheticDataNames[i], "Case: ")
    print("Sequential Search: ")
    SEQ = InsertTime2(SequentialSearchSet, SyntheticDataList[i])
    SEQSyn.append(SEQ)
    print("Binary Search Tree: ")
    BST = InsertTime2(BinarySearchTreeSet, SyntheticDataList[i])
    BSTSyn.append(BST)
    print("Balanced Search Tree: ")
    RBT = InsertTime2(BalancedSearchTreeSet, SyntheticDataList[i])
    RBTSyn.append(RBT)
    print("Bloom Filter: ")
    BT = BloomFilterSet()
    BT.initVariables(len(SyntheticDataList[i]), 0.01, hashFunctions)
    InsertStart = timeit.default_timer()
    for word in SyntheticDataList[i]:
        BT.insertElement(word)
    InsertEnd = timeit.default_timer()
    InsertTime = InsertEnd - InsertStart
    print("Insert time taken is :", InsertTime)
    BTSyn.append(BT)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

For Normal Case: 
Sequential Search: 
Insert time taken is : 0.0008363000001736509
Binary Search Tree: 
Insert time taken is : 0.01195640000014464
Balanced Search Tree: 
Insert time taken is : 0.037257400000271446
Bloom Filter: 
Insert time taken is : 0.02174609999974564
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
For Long String Case: 
Sequential Search: 
Insert time taken is : 0.000693299999966257
Binary Search Tree: 
Insert time taken is : 0.011469499999748223
Balanced Search Tree: 
Insert time taken is : 0.03685989999985395
Bloom Filter: 
Insert time taken is : 0.09174159999975018
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
For Special String Case: 
Sequential Search: 
Insert time taken is : 0.0007998000000952743
Binary Search Tree: 
Insert time taken is : 0.010834900000190828
Balanced Search Tree: 
Insert time taken is : 0.03668870000001334
Bloom Filter: 
Insert time taken is : 0.02130570000008447
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
For Ascending Order Ca

TypeError: '<' not supported between instances of 'NoneType' and 'str'

### Search Time Taken

In [ ]:
def SearchTime2(obj, searchdata):
    SearchStart = timeit.default_timer()
    for ws in searchdata:
        obj.searchElement(ws)
    SearchEnd = timeit.default_timer()
    SearchTime = SearchEnd - SearchStart
    print("Search time taken is :", SearchTime)

In [ ]:
for i in range (len(synSearchList)):
    print("For", SyntheticDataNames[i], "Case: ")
    print("Sequential Search: ")
    SEQ = SearchTime2(SEQSyn[i], synSearchList[i])
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Binary Search Tree: ")
    BST = SearchTime2(BSTSyn[i], synSearchList[i])
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Balanced Search Tree: ")
    RBT = SearchTime2(RBTSyn[i], synSearchList[i])
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Bloom Filter: ")
    BT = SearchTime2(BTSyn[i], synSearchList[i])
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("                                                 ")

For Normal Case: 
Sequential Search: 
Search time taken is : 0.011950699999943026
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Binary Search Tree: 
Search time taken is : 0.0011890999994648155
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Balanced Search Tree: 
Search time taken is : 0.0007524999991801451
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Bloom Filter: 
Search time taken is : 0.004684200000156125
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                                                 
For Long String Case: 
Sequential Search: 
Search time taken is : 0.01957239999956073
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Binary Search Tree: 
Search time taken is : 0.0010870000005525071
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Balanced Search Tree: 
Search time taken is : 0.0007821999997759121
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Bloom Filter: 
Search time taken is : 0.03139980000014475
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

IndexError: list index out of range

# --------

In [ ]:
rbt = BalancedSearchTreeSet()
rbt.insertElement('b')
rbt.insertElement('a')
rbt.insertElement('c')

print(rbt.searchElement('b'))
print(rbt.searchElement('a'))
print(rbt.searchElement('c'))

True
True
True


Sequential Search

In [ ]:
# for key in files.keys():
#     seqInsertStart = timeit.default_timer()
#     for word in files[key]:
#         SEQs.insertElement(word)
#     seqInsertEnd = timeit.default_timer()
#     seqInsertTime = seqInsertEnd - seqInsertStart
#     print("Insert time taken for ", key, "is : ", seqInsertTime)

AttributeError: 'list' object has no attribute 'insertElement'

Sequential Search

In [ ]:
SeqSearchStart = timeit.default_timer()
for ws in wordsforsearch:
    SEQs.searchElement(ws)
SeqSearchEnd = timeit.default_timer()
SeqSearchTime = SeqSearchEnd - SeqSearchStart
print("Search time taken for 'test-search.txt' is : ", SeqSearchTime)

Binary Search Tree

In [ ]:
for key in files.keys():
    binaryInsertStart = timeit.default_timer()
    for word in files[key]:
        BSTs.insertElement(word)
    binaryInsertEnd = timeit.default_timer()
    
    binaryInsertTime = binaryInsertEnd - binaryInsertStart
    print("Insert time taken for ", key, "is : ", binaryInsertTime)

AttributeError: 'list' object has no attribute 'insertElement'

Binary Search Tree

In [ ]:
BinarySearchStart = timeit.default_timer()
for ws in wordsforsearch:
    BSTs.searchElement(ws)
BinarySearchEnd = timeit.default_timer()
BinarySearchTime = BinarySearchEnd - BinarySearchStart
print("Search time taken for 'test-search.txt' is : ", BinarySearchTime)

AttributeError: 'list' object has no attribute 'searchElement'

Balanced Search Tree

In [ ]:
for key in files.keys():
    RedBlackInsertStart = timeit.default_timer()
    for word in files[key]:
        RBT.insertElement(word)
    RedBlackInsertEnd = timeit.default_timer()
    RedBlackInsertTime = RedBlackInsertEnd - RedBlackInsertStart
    print("Insert time taken for ", key, "is : ", RedBlackInsertTime)

Insert time taken for  'test1-mobydick.txt' is :  2.1321972999994614
Insert time taken for  'test2-warpeace.txt' is :  6.030556100000467
Insert time taken for  'test-search.txt' is :  60.04795980000017


Balanced Search Tree

In [ ]:
RedBlackSearchStart = timeit.default_timer()
for ws in wordsforsearch:
    RBT.searchElement(ws)
RedBlackSearchEnd = timeit.default_timer()
RedBlackSearchTime = RedBlackSearchEnd - RedBlackSearchStart
print("Search time taken for 'test-search.txt' is : ", RedBlackSearchTime)

Search time taken for 'test-search.txt' is :  0.0031930999994074227


Bloom Filter

In [ ]:
for key in files.keys():
    BFInsertStart = timeit.default_timer()
    for word in files[key]:
        BF.insertElement(word)
    BFInsertEnd = timeit.default_timer()
    BFInsertTime = BFInsertEnd - BFInsertStart
    print("Insert time taken for ", key, "is : ", BFInsertTime)

Insert time taken for  'test1-mobydick.txt' is :  2.006846799999039
Insert time taken for  'test2-warpeace.txt' is :  5.462229099999604
Insert time taken for  'test3-dickens.txt' is :  49.96937220000109


Bloom Filter

In [ ]:
BFSearchStart = timeit.default_timer()
for word in wordsforsearch:
    BF.searchElement(word)
BFSearchEnd = timeit.default_timer()
BFSearchTime = BFSearchEnd - BFSearchStart
print("Search time taken for 'test-search.txt' is : ", BFSearchTime)

Search time taken for 'test-search.txt' is :  0.005908900000576978


In [ ]:
a = './data/testfiles/test1-mobydick.txt'
b = './data/testfiles/test2-warpeace.txt'
c = './data/testfiles/test3-dickens.txt'
insertFiles = [a,b,c]

for file in insertFiles:
    seqInsertStart = timeit.default_timer()
    seqS.insertElement(file)
    seqInsertEnd = timeit.default_timer()
    seqInsertTime = seqInsertEnd - seqInsertStart
    print(seqInsertTime)

In [ ]:
a = './data/testfiles/test1-mobydick.txt'
b = './data/testfiles/test2-warpeace.txt'
c = './data/testfiles/test3-dickens.txt'
d = "./data/testfiles/test-search.txt"
insertFiles = [a,b,c]
for file in insertFiles:
    seqS.insertElement(file)
    SeqSearchStart = timeit.default_timer()
    seqS.searchElement(d)
    SeqSearchEnd = timeit.default_timer()
    SeqSearchTime = SeqSearchEnd - SeqSearchStart
    print(SeqSearchTime)

False ,['zero'] is not found
94.96285220000027
False ,['zero'] is not found
106.49899989999994
False ,['zero'] is not found
183.5646872999996


In [ ]:
a = './data/testfiles/test1-mobydick.txt'
b = './data/testfiles/test2-warpeace.txt'
c = './data/testfiles/test3-dickens.txt'
insertFiles = [a,b,c]

for file in insertFiles:
    BSTInsertStart = timeit.default_timer()
    rbt.insertElement(file)
    BSTInsertEnd = timeit.default_timer()
    BSTInsertTime = BSTInsertEnd - BSTInsertStart
    print(BSTInsertTime)


1.3200000012147939e-05
1.6899999991437653e-05
4.79999994240643e-06


In [ ]:
def insert_file():
    a = './data/testfiles/test1-mobydick.txt'
    b = './data/testfiles/test2-warpeace.txt'
    c = './data/testfiles/test3-dickens.txt'
    insertFiles = [a,b,c]
    for file in insertFiles:
        BSTInsertStart = timeit.default_timer()
        bst.insertElement(file)
        BSTInsertEnd = timeit.default_timer()
        BSTInsertTime = BSTInsertEnd - BSTInsertStart
        print(BSTInsertTime)

def insert_real_data_test(words, bst):
    BSTInsertStart = timeit.default_timer() 
    for word in words:
        bst.insertElement(word)
    BSTInsertEnd = timeit.default_timer()
    BSTInsertTime = BSTInsertEnd - BSTInsertStart
    print("Insert time: ", BSTInsertTime)

def search_words_in_set(words, bst):
    BSTSearchStart = timeit.default_timer() 
    for word in words:
        bst.searchElement(word)
    BSTSearchEnd = timeit.default_timer()
    BSTSearchTime = BSTSearchEnd - BSTSearchStart
    print("Search time (word already in set): ", BSTSearchTime)


def search_words(words, bst):
    BSTSearchStart = timeit.default_timer() 
    for word in words:
        bst.searchElement(word)
    BSTSearchEnd = timeit.default_timer()
    BSTSearchTime = BSTSearchEnd - BSTSearchStart
    print("Search time (test-search words): ", BSTSearchTime)


bst1 = BinarySearchTreeSet()
bst2 = BinarySearchTreeSet()
bst3 = BinarySearchTreeSet()

insert_real_data_test(words, bst1)
insert_real_data_test(words2, bst2)
insert_real_data_test(words3, bst3)

search_words_in_set(words, bst1)
search_words_in_set(words2, bst2)
search_words_in_set(words3, bst3)

search_words(words, bst1)
search_words(words2, bst2)
search_words(words3, bst3)

In [ ]:
a = './data/testfiles/test1-mobydick.txt'
b = './data/testfiles/test2-warpeace.txt'
c = './data/testfiles/test3-dickens.txt'
d = "./data/testfiles/test-search.txt"
insertFiles = [a,b,c]

for file in insertFiles:
    rbt.insertElement(file)
    BSTSearchStart = timeit.default_timer()
    rbt.searchElement(d)
    BSTSearchEnd = timeit.default_timer()
    BSTSearchTime = BSTSearchEnd - BSTSearchStart
    print(BSTSearchTime)

2.4000000848900527e-06
8.000001798791345e-07
7.000003279244993e-07


In [ ]:
# 1st version

class TestDataGenerator():

    def __init__(self):
            self.syntheticData = []
            
            pass           
            
    # sythetic data -- integers:
    def generateNum(self, size):
        for _ in range(size):
            item = random.randrange(0,1000000)
            self.syntheticData.append(item)
        print(self.syntheticData) # delete when it's finished
        return self.syntheticData   
    
    # sythetic data -- 4-letter string (randomly without real meanings):
    def generateString(self,size):
        letters = string.ascii_lowercase
        for letter in letters:
            for _ in range(size):
                letter1 = ''.join(random.choice(letters))
                letter2 = ''.join(random.choice(letters))
                letter3 = ''.join(random.choice(letters))
                stringGenerated = letter + letter1 + letter2 + letter3
                self.syntheticData.append(stringGenerated)
        print(self.syntheticData) # delete when it's finished
        return self.syntheticData

toGenerate = TestDataGenerator()
toSearch = TestDataGenerator()

toGenerate.generateString(1)
toSearch.generateNum(10)

['aiet', 'bbrr', 'csgv', 'dmux', 'ezat', 'fxew', 'gfhm', 'hyjd', 'iaby', 'jwzk', 'kshf', 'lwrx', 'mdrk', 'nbfd', 'ogxs', 'pgyh', 'qyvb', 'rxeo', 'sjff', 'tosj', 'ukjl', 'vhgj', 'wlgn', 'xtyg', 'yvhy', 'zchc']
[634667, 496119, 767303, 410927, 353639, 23654, 399978, 520985, 373862, 726654]


[634667, 496119, 767303, 410927, 353639, 23654, 399978, 520985, 373862, 726654]